## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [ ]:
# Create a list of at least 10 column names to use as X data
columns = ['Age', 
        'HourlyRate',
        'DistanceFromHome',
        'Education',
        'JobLevel',
        'JobSatisfaction',
        'NumCompaniesWorked',
        'OverTime',
        'PercentSalaryHike',
        'PerformanceRating',
        'StockOptionLevel',
        'YearsAtCompany',
        'YearsInCurrentRole',
        'YearsSinceLastPromotion',
        'YearsWithCurrManager'
        ]


# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.head()


In [ ]:
X_df['OverTime'] = X_df['OverTime'].replace({'Yes': 1, 'No': 0})
X_df['OverTime'].value_counts()

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

X_train.dtypes

In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# X_train['OverTime'] = X_train['OverTime'].replace({'Yes': 1, 'No': 0})
# X_train['OverTime'].value_counts()


In [ ]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
scaler.transform(X_train)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_encoder.transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])

print(f"y_train_department.shape: {y_train_department.shape} , y_test_department.shape: {y_test_department.shape}")

In [ ]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

print(f"y_train_attrition.shape: {y_train_attrition.shape} , y_test_attrition.shape: {y_test_attrition.shape}")

## Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
input_nodes = X_train.columns

# Create the input layer
input = tf.keras.layers.Input(shape=(len(input_nodes),), name='input')

# Create at least two shared layers
shared1 = tf.keras.layers.Dense(64, activation='relu', name='shared1')(input)
shared2 = tf.keras.layers.Dense(128, activation='relu', name='shared2')(shared1)


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = tf.keras.layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = tf.keras.layers.Dense(3, activation='relu', name='department_output')(department_hidden)

In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = tf.keras.layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = tf.keras.layers.Dense(2, activation='relu', name='attrition_output')(attrition_hidden)


In [ ]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

In [ ]:
# Train the model
model.fit(
    X_train, 
    {'department_output': y_train_department, 
     'attrition_output': y_train_attrition}, 
    epochs=10, 
    validation_split=0.2, 
    verbose=2
)


In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'department_output': y_test_department, 'attrition_output': y_test_attrition})
test_results

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]*100:.2f}")
print(f"Attrition Accuracy: {test_results[4]*100:.2f}")


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 